# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))


1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')
nltk.download('punkt')
from string import punctuation

def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub('http\S+', ' ', text) 
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    for r in re.findall(r'\$\S+',text):
        text = text.replace(r, ' ') 
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    for r in re.findall(r'\@\S+',text):
        text = text.replace(r, ' ')
        
    # Replace everything not a letter with a space
    text = ''.join([c for c in text if c not in punctuation])
            
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = nltk.tokenize.word_tokenize(text)
    
    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(item) for item in tokens]
       
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement
tokenized = []
for message in messages:
    tokenized.extend(preprocess(message))
    
print(len(tokenized))

17185576


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter

"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
token_count = Counter(tokenized)
#print(list(token_count.items())[:-3])


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 
# sorting the words from most to least frequent in text occurrence
# sorted_count = sorted(token_count, key=token_count.get, reverse=True)
# print((token_count['the'])) 
# print((sorted_count[0]))
# print((sorted_count[280182]))
#print((token_count['earnings']))

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {word:count/len(tokenized) for word, count in token_count.items()}
# k = Counter(freqs)
# print(k.most_common(10))

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 50/len(tokenized)

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 25000/len(tokenized)

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word for word in freqs if freqs[word] > high_cutoff]

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
#print(K_most_common)
print(filtered_words[:20]) 


['great', 'wait', 'staanalystalert', 'jefferies', 'maintains', 'rating', 'hold', 'setting', 'target', 'usd', '35000', 'our', 'own', 'verdict', 'heard', 'there', 'guy', 'who', 'know', 'someone']


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [9]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement
# sorting the words from most to least frequent in text occurrence
sorted_count = sorted (filtered_words, key = token_count.get, reverse= True)

# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {ii:word for word, ii in enumerate(sorted_count)}

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word:ii for word, ii in id2vocab.items()}

In [10]:
# tokenized with the words not in `filtered_words` removed.
filtered = []
for message in messages:
    cleansed_message = preprocess(message)
    filtered.append([word for word in message.split() if word in filtered_words])

In [11]:
# import pickle

# with open('filtered_data.pkl', 'wb') as f:
#     pickle.dump(filtered, f)

In [12]:
# import pickle
# with open('filtered_data.pkl', 'rb') as f:
#     filtered = pickle.load(f)

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [13]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [14]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1766607776020607

Finally let's convert our tokens into integer ids which we can pass to the network.

In [15]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']


## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [16]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout_prob = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(self.embed_size, lstm_size, lstm_layers, dropout, batch_first= True)
        self.dropout = nn.Dropout(self.dropout_prob)
        self.fc = nn.Linear(lstm_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        # TODO Implement 
        weights = next(self.parameters()).data
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        if(train_on_gpu):
            hidden = (weights.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
            (weights.new(self.lstm_layers, batch_size, self.lstm_size)).zero_().cuda())
        else:
            hidden = (weights.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
            (weights.new(self.lstm_layers, batch_size, self.lstm_size)).zero_())
        
        return hidden  

    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        batch_size = nn_input.shape[0]
        embed_out = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embed_out, hidden_state)
        lstm_out = lstm_out.contiguous().view (-1, self.lstm_size)
        drop_out = self.dropout(lstm_out)
        fc_out = self.fc(drop_out)
        logps = self.softmax(fc_out)
        # reshape to be batch_size first
        logps = logps.view(batch_size, -1)
        logps = logps[:,-self.output_size:]
        return logps, hidden_state

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### View Model

In [18]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
print(model)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)
input = (torch.randint(0, 1000, (5, 4), dtype=torch.int64)).to(device)
hidden = model.init_hidden(5)

logps, h = model.forward(input, hidden)
print(logps)

TextClassifier(
  (embedding): Embedding(10535, 10)
  (lstm): LSTM(10, 6, num_layers=2, bias=0.1, batch_first=True)
  (dropout): Dropout(p=0.1)
  (fc): Linear(in_features=6, out_features=5, bias=True)
  (softmax): LogSoftmax()
)
tensor([[-1.7900, -1.9440, -1.4496, -1.3112, -1.6834],
        [-1.7962, -1.9474, -1.4523, -1.3073, -1.6773],
        [-1.7976, -1.9421, -1.4340, -1.3047, -1.7077],
        [-1.7956, -1.9327, -1.4171, -1.3004, -1.7465],
        [-1.8023, -1.9362, -1.4172, -1.3010, -1.7362]], device='cuda:0')


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [19]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = (len(messages)//batch_size) * batch_size
    
    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros(( len(batch_messages),sequence_length), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[ batch_num, start_idx:] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [20]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   
split_size = 0.8

# TODO Implement 
print(token_ids[:10])
train_set = int(split_size * len(token_ids))
valid_set = (1 - split_size)/2 * len(token_ids)
train_features = token_ids[:train_set]
valid_features = token_ids[train_set:]
train_labels = sentiments[: train_set ]
valid_labels = sentiments[train_set : ]

[[53, 143], [84, 500, 68, 186, 523], [1225, 110, 63, 372, 63, 1478, 318], [1553, 281, 288], [91, 220, 243, 548], [429, 811], [18, 47], [51, 66, 38, 37, 26], [226, 410, 9], [679, 13, 30, 816, 328, 3745, 236]]


In [21]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
text_batch, labels = text_batch.to(device), labels.to(device)
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
model.to(device)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [22]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(10536, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, bias=0.2, batch_first=True)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (softmax): LogSoftmax()
)

In [23]:
import numpy as np

"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 5
batch_size = 500
learning_rate = 0.001
clip = 5

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    steps = 0
    
    hidden = model.init_hidden(batch_size)
    
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=200, shuffle=True):
        steps += 1
        
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
                
        # TODO Implement: Train Model
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        hidden = tuple([each.data for each in hidden])

        # zero accumulated gradients
        model.zero_grad()

        # get the output from the model
        output, hidden = model(text_batch, hidden)
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels)
        
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if steps % print_every == 0:
            model.eval()
            # Get validation loss
            val_h = model.init_hidden(batch_size)
            val_losses = []
            
            for val_batch, labels in dataloader(
            valid_features, valid_labels, batch_size=batch_size, sequence_length=200, shuffle=True):
                #print('val_batch:' ,val_batch.shape)
                #print('labels:' ,labels)
                #print('val_h:' ,val_h)
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                # Set Device
                val_batch, labels = val_batch.to(device), labels.to(device)
               
                output, val_h = model(val_batch, val_h)
                
                val_loss = criterion(output.squeeze(), labels)

                val_losses.append(val_loss.item())
            
            # TODO Implement: Print metrics
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(steps),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            
            model.train()

Starting epoch 1
Epoch: 1/5... Step: 100... Loss: 1.250681... Val Loss: 1.341414
Epoch: 1/5... Step: 200... Loss: 1.154449... Val Loss: 1.293771
Epoch: 1/5... Step: 300... Loss: 1.172062... Val Loss: 1.281225
Epoch: 1/5... Step: 400... Loss: 1.274344... Val Loss: 1.266462
Epoch: 1/5... Step: 500... Loss: 1.217227... Val Loss: 1.258506
Epoch: 1/5... Step: 600... Loss: 1.230626... Val Loss: 1.260411
Epoch: 1/5... Step: 700... Loss: 1.141083... Val Loss: 1.258243
Epoch: 1/5... Step: 800... Loss: 1.191902... Val Loss: 1.246843
Epoch: 1/5... Step: 900... Loss: 1.217206... Val Loss: 1.242405
Epoch: 1/5... Step: 1000... Loss: 1.167990... Val Loss: 1.246399
Epoch: 1/5... Step: 1100... Loss: 1.126013... Val Loss: 1.240401
Epoch: 1/5... Step: 1200... Loss: 1.100687... Val Loss: 1.234603
Epoch: 1/5... Step: 1300... Loss: 1.207977... Val Loss: 1.239033
Epoch: 1/5... Step: 1400... Loss: 1.143063... Val Loss: 1.239441
Starting epoch 2
Epoch: 2/5... Step: 100... Loss: 1.117396... Val Loss: 1.258987
E

KeyboardInterrupt: 

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [24]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
        
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in vocab]
    
    # Convert words to ids
    tokens = np.asarray([vocab[word] for word in tokens])
    
    # Adding a batch dimension
    text_input = torch.from_numpy(tokens).unsqueeze(dim=0).to(device)
    print(torch.from_numpy(tokens))
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [25]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to(device)
predict(text, model, vocab)


tensor([  635,   795,  2408,  1676,   914,   378,     0])


tensor([[ 0.0000,  0.0119,  0.0057,  0.9782,  0.0041]], device='cuda:0')

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**


The prediction of the model for the above text is 97% positive which is better than the random predcition of just 20 %

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [26]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [27]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [28]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [29]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

tensor([  579,  3299,   750,   353,   669,   611])
tensor([ 576,   79,  148,   75,   42])
tensor([ 1360])


{'symbol': '$AAPL',
 'score': tensor([[ 0.6171,  0.0231,  0.0093,  0.0230,  0.3274]], device='cuda:0'),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.